In [1]:
import cv2
import os, sys
import json
import time
import skimage.io
import configparser
import numpy as np
import tensorflow as tf

In [2]:
def getSectionToUse(config_dict):
    ini_section = 'DEFAULT'
    if config_dict.has_option('DEFAULT', 'section_to_use'):
        if config_dict['DEFAULT']['section_to_use'] != '':
            ini_section = config_dict['DEFAULT']['section_to_use']
            if ini_section not in config_dict.sections():
                raise ValueError('section_to_use  ' + ini_section + ' stated in env.ini is not a valid section.')
    else:
        print('Section_to_use not specified in env.ini. Using Default values.')
    return ini_section

config_dict = configparser.ConfigParser()
config_dict.read('env.ini')
ini_section = getSectionToUse(config_dict)
RCNN_DIR = config_dict[ini_section]['RCNN_DIR']
if not os.path.isdir(RCNN_DIR):
    raise NotADirectoryError(RCNN_DIR + " in env.ini is not a directory.")


# SET UP RCNN LOGS AND COCO DEPENDENCIES 
# Directory to save logs and trained model
MODEL_DIR = os.path.join(RCNN_DIR, "logs")
print('Creating model')
# Local path to trained weights file
COCO_MODEL_PATH = os.path.join(RCNN_DIR, "mask_rcnn_coco.h5")
# Download COCO trained weights from Releases if needed
if not os.path.exists(COCO_MODEL_PATH):
    utils.download_trained_weights(COCO_MODEL_PATH)
    
sys.path.append(os.path.abspath(RCNN_DIR))
from mrcnn import utils
import mrcnn.model as modellib
from mrcnn import visualize
# Import COCO config
sys.path.append(os.path.join(RCNN_DIR, "samples/coco/"))  # To find local version
import coco

def detectColour(image, aoi, threshold=0.3 ):
    # define the list of boundaries
    boundaries = [ #RGB
        ([100, 50, 50], [255, 130, 130], 'RED'),
        ([50, 100, 50], [130, 255, 130], 'GREEN'),
        ([50, 50, 100], [130, 130, 255], 'BLUE'),
        ([100, 100, 50], [255, 255, 130], 'YELLOW' ),
        ([100, 100, 100], [225, 225, 225], 'SILVER' ), 
        ([225, 225, 225], [255, 255, 255], 'WHITE' ),
        ([0, 0, 0], [100, 100, 100], 'BLACK' ) 
    ]
    # loop over the boundaries
    for (lower, upper, colour) in boundaries:
    # (lower, upper, colour) = boundaries[4]
        # create NumPy arrays from the boundaries
        lower = np.array(lower, dtype = "uint8")
        upper = np.array(upper, dtype = "uint8")

        # find the colors within the specified boundaries and apply
        # the mask
        mask = cv2.inRange(image, lower, upper)

        tmp_mask = np.logical_and(mask, aoi)
        output = cv2.bitwise_and(image, image, mask = tmp_mask.astype("uint8"))
        # print(np.sum(np.logical_and(mask, aoi))/np.sum(aoi))
        isColour = np.sum(np.logical_and(mask, aoi))/np.sum(aoi) > threshold
        if isColour:
            return colour, output
            
    return "OTHERS", output

def getImageIdfrmPath(path, case='oneMotoring'):
    image_id = ''
    # for onemotoring:
    if case.lower() == 'oneMotoring'.lower():
        path, im = os.path.split(path)
        path, date = os.path.split(path)
        path, location = os.path.split(path)
        image_id = location.replace(' ', '_') + '-' + date + '-' + im.replace('.jpeg','')
    return image_id


def initializeVariables(config_dict):
    ini_section = getSectionToUse(config_dict)
    # Initialise of output<dict> with items in reference.json
    F_ref = open(config_dict[ini_section]['reference_file'],'r')  # TODO: add error checks
    text_ = F_ref.read()
    json_ref = json.loads(text_)
    jsonOutput = {'categories': json_ref['categories'],                  'annotations': [],                  'images': json_ref['images'],                  'licenses': json_ref['licenses']}
    F_ref.close()
    
    if not config_dict.has_option(ini_section, 'obj_of_interest'):
        raise ValueError('Object of interest not found in env.ini')
    
    if not config_dict.has_option(ini_section, 'im_dir'):
        raise ValueError('Image directory not found in env.ini')
        
    if not config_dict.has_option(ini_section, 'output_file'):
        raise ValueError('Output_file path not found in env.ini')
    
    if not config_dict.has_option(ini_section, 'batch_size'):
        raise ValueError('batch_size not found in env.ini')
    
    if not config_dict.has_option(ini_section, 'threshold'):
        raise ValueError('Min. size for object detection, threshold, not found in env.ini')
        
    if not config_dict.has_option(ini_section, 'ann_id_prefix'):
        raise ValueError('Annotation id prefix, ann_id_prefix not found in env.ini')
    return jsonOutput, config_dict[ini_section]

def get_lines_as_list(file_path):
    File = open(file_path, 'r')
    lines = File.readlines()
    lines_in_list = [x.strip() for x in lines] 
    File.close()
    return lines_in_list


Creating model


Using TensorFlow backend.


In [3]:
# INITIALISATION OF VARIABLES
jsonOutput, config_ = initializeVariables(config_dict)  # use config with underscore to prevent conflict with pyco's config
OBJECTS_OF_INTEREST_ls = config_['obj_of_interest']
IM_DIR = config_['im_dir']
output_file = config_['output_file']
BATCH_SIZE = int(config_['batch_size'])
THRESHOLD = float(config_['threshold'])
ANN_ID_PREFIX = config_['ann_id_prefix']
OBJECTS_OF_INTEREST_ls = [ int(x) for x in OBJECTS_OF_INTEREST_ls.replace('[','').replace(']','').split(',')]
done_list_path = config_['done_list']
'BUS' in ', '.join([items['id'] for items in jsonOutput['categories']]).split(', ')
ann_json_list = []
if len(done_list_path) != 0:
    done_list = get_lines_as_list(done_list_path)
else:
    done_list = []
global_counter = 1
other_list = []


# In[9]:


# COCO Class names
# Index of the class in the list is its ID. For example, to get ID of
# the teddy bear class, use: class_names.index('teddy bear')
# class_names = ['BG', 'person', 'bicycle', 'car', 'motorcycle', 'airplane',
#                'bus', 'train', 'truck', 'boat', 'traffic light',
#                'fire hydrant', 'stop sign']

# OVERRIDES MASK_RCNN DEFAULTS
class_names = ['BG', 'person', 'bicycle', 'SEDAN', 'motorcycle', 'airplane',
               'bus', 'train', 'truck', 'boat', 'traffic light',
               'fire hydrant', 'stop sign']

# REMOVES OBJECTS THAT ARE OF INTEREST BUT ARE NOT IN REFERENCE CATEGORY LIST
categories_list = ', '.join([items['id'] for items in jsonOutput['categories']]).lower().split(', ')
ref_indices_to_be_removed = []
for obj_ref_idx in OBJECTS_OF_INTEREST_ls:
    if not class_names[obj_ref_idx].lower() in categories_list:
        ref_indices_to_be_removed.append(obj_ref_idx)
for i in ref_indices_to_be_removed:
    print('Removing ' + class_names[i] + ' from objects-of-interest list as it is not present in the category list of reference.json')
    OBJECTS_OF_INTEREST_ls.remove(i)


# In[10]:


# Directory to save logs and trained model
MODEL_DIR = os.path.join(RCNN_DIR, "logs")

# Local path to trained weights file
COCO_MODEL_PATH = os.path.join(RCNN_DIR, "mask_rcnn_coco.h5")
# Download COCO trained weights from Releases if needed
if not os.path.exists(COCO_MODEL_PATH):
    utils.download_trained_weights(COCO_MODEL_PATH)

class InferenceConfig(coco.CocoConfig):
    # Set batch size to 1 since we'll be running inference on
    # one image at a time. Batch size = GPU_COUNT * IMAGES_PER_GPU
    GPU_COUNT = 1
    IMAGES_PER_GPU = BATCH_SIZE

inf_config = InferenceConfig()
inf_config.display()


# In[ ]:

list_of_image_paths = []
assert os.path.isdir(IM_DIR), IM_DIR + " is an invalid image directory."

for path, subdirs, files in os.walk(IM_DIR):
    for file in files:
        if file.endswith('.jpeg'):
            list_of_image_paths.append(os.path.join(path, file))

Removing bicycle from objects-of-interest list as it is not present in the category list of reference.json
Removing train from objects-of-interest list as it is not present in the category list of reference.json
Removing traffic light from objects-of-interest list as it is not present in the category list of reference.json
Removing stop sign from objects-of-interest list as it is not present in the category list of reference.json

Configurations:
BACKBONE                       resnet101
BACKBONE_STRIDES               [4, 8, 16, 32, 64]
BATCH_SIZE                     2
BBOX_STD_DEV                   [0.1 0.1 0.2 0.2]
COMPUTE_BACKBONE_SHAPE         None
DETECTION_MAX_INSTANCES        100
DETECTION_MIN_CONFIDENCE       0.7
DETECTION_NMS_THRESHOLD        0.3
FPN_CLASSIF_FC_LAYERS_SIZE     1024
GPU_COUNT                      1
GRADIENT_CLIP_NORM             5.0
IMAGES_PER_GPU                 2
IMAGE_CHANNEL_COUNT            3
IMAGE_MAX_DIM                  1024
IMAGE_META_SIZE              

True

Creating model


Using TensorFlow backend.
